In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import Qiskit Nature and PySCF components
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

# Import PySCF components for classical calculations
from pyscf import gto, scf, fci


def get_oh_coords(d):
    """
    Returns O-H linear geometry at a given bond distance d (Å).
    This function was adapted to define the OH- molecule.
    """
    return [["O", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, d]]]


def get_pyscf_mol(coords, charge, spin, basis):
    """Builds a PySCF Mole object."""
    mol = gto.Mole()
    mol.atom = coords
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol


def get_qiskit_problem(coords, charge, spin, basis):
    """Generates a Qiskit Nature ElectronicStructureProblem."""
    # Format atom string for PySCFDriver
    atom_str = "; ".join([f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}" for atom in coords])
    driver = PySCFDriver(atom=atom_str, unit=DistanceUnit.ANGSTROM,
                         charge=charge, spin=spin, basis=basis)
    return driver.run()


def run_uhf_pyscf(mol):
    """Runs a classical UHF calculation with PySCF."""
    mf = scf.UHF(mol)
    ehf = mf.kernel()
    return ehf


def run_fci_pyscf(mol):
    """Runs a classical Full CI calculation with PySCF (exact solution for the basis)."""
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci


def run_vqe(problem, mapper):
    """Runs the VQE algorithm to find the ground state energy."""
    estimator = Estimator()
    optimizer = COBYLA(maxiter=1500)

    # Set up the initial state and ansatz (UCCSD)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)

    # Initialize and run the VQE algorithm
    vqe = VQE(estimator, ansatz, optimizer, initial_point=[0] * ansatz.num_parameters)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    result = vqe.compute_minimum_eigenvalue(qubit_op)

    # Interpret and return the final energy
    return problem.interpret(result).total_energies[0].real


# === Main Setup ===
# Define the molecular parameters and basis set.
# OH- is a closed-shell molecule with a charge of -1.
basis = 'sto3g'
charge = -1
spin = 0
# Define the range of bond distances to scan.
# The equilibrium bond distance for OH- is ~0.97 Å. This range is suitable.
dists = np.linspace(0.5, 3.0, 50)

vqe_energies, uhf_energies, fci_energies = [], [], []

print("Starting calculations for OH- molecule...")
print("-" * 50)
print(f"{'Distance (Å)':<15} | {'VQE Energy (Ha)':<20} | {'UHF Energy (Ha)':<20} | {'FCI Energy (Ha)':<20}")
print("-" * 50)

# === Calculation Loop ===
# Iterate over each bond distance, calculate energies, and store the results.
for i, d in enumerate(dists):
    # Get the coordinates for the current distance
    coords = get_oh_coords(d)
    mol = get_pyscf_mol(coords, charge, spin, basis) # PySCF mol for classical methods

    # VQE Calculation
    try:
        problem = get_qiskit_problem(coords, charge, spin, basis)
        mapper = ParityMapper(num_particles=problem.num_particles)
        vqe_energy = run_vqe(problem, mapper)
    except Exception as e:
        vqe_energy = np.nan
        print(f"VQE failed at d={d:.2f}: {e}")

    # UHF Calculation
    try:
        uhf_energy = run_uhf_pyscf(mol)
    except Exception as e:
        uhf_energy = np.nan
        print(f"UHF failed at d={d:.2f}: {e}")

    # FCI Calculation
    try:
        fci_energy = run_fci_pyscf(mol)
    except Exception as e:
        fci_energy = np.nan
        print(f"FCI failed at d={d:.2f}: {e}")

    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)

    # Print progress
    print(f"{d:<15.2f} | {vqe_energy:<20.6f} | {uhf_energy:<20.6f} | {fci_energy:<20.6f}")

print("-" * 50)
print("Calculations complete.")

# === Plotting ===
plt.figure(figsize=(8, 6))
plt.plot(dists, vqe_energies, label='VQE (UCCSD)', color='limegreen')
plt.plot(dists, uhf_energies, label='UHF (Classical)', color='red')
plt.plot(dists, fci_energies, '--', label='FCI (Exact)', color='black')

# Updated labels for OH- molecule
plt.xlabel("O-H Distance (Å)", fontsize=12)
plt.ylabel("Energy (Hartree)", fontsize=12)
plt.title("OH⁻ Potential Energy Curve (STO-3G basis)", fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


Starting calculations for OH- molecule...
--------------------------------------------------
Distance (Å)    | VQE Energy (Ha)      | UHF Energy (Ha)      | FCI Energy (Ha)     
--------------------------------------------------


/tmp/ipykernel_550933/3125971715.py:62: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


converged SCF energy = -73.1231976450769  <S^2> = 1.438849e-13  2S+1 = 1
converged SCF energy = -73.123197645077
0.50            | -73.130042           | -73.123198           | -73.130042          
converged SCF energy = -73.4157104359657  <S^2> = 3.7125858e-13  2S+1 = 1
converged SCF energy = -73.415710435966
0.55            | -73.423850           | -73.415710           | -73.423851          
converged SCF energy = -73.6208160972525  <S^2> = 3.5527137e-15  2S+1 = 1
converged SCF energy = -73.6208160972526
0.60            | -73.630358           | -73.620816           | -73.630359          
converged SCF energy = -73.7649057112325  <S^2> = 1.1546319e-14  2S+1 = 1
converged SCF energy = -73.7649057112326
0.65            | -73.775990           | -73.764906           | -73.775991          
converged SCF energy = -73.8661220314491  <S^2> = 4.0856207e-14  2S+1 = 1
converged SCF energy = -73.8661220314492
0.70            | -73.878912           | -73.866122           | -73.878913          
con

In [2]:
import numpy as np
import time
from datetime import datetime

# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports (Modern API) ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports (Modern API) ---
from qiskit_algorithms import VQE, AdaptVQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator

# ----- Configuration -----
BASIS_SET = "sto-3g"
BOND_LENGTH = 1.087  # Angstrom, experimental equilibrium C-H bond length for CH₄

# ----- Main Calculation Function -----
def main():
    """
    Calculates the ground state energy of CH₄ using four different methods:
    UHF, FCI, standard VQE, and AdaptVQE.
    """
    print(f"CH₄ Ground State Calculation Comparison for Basis: {BASIS_SET}")
    print(f"Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Define the molecule's geometry as a tetrahedral structure
    # Using standard coordinates for a tetrahedron
    r_xy = BOND_LENGTH * np.sqrt(8/9)
    z_h = -BOND_LENGTH / 3.0
    
    atom_str = (
        f"C 0.0 0.0 0.0; "
        f"H 0.0 0.0 {BOND_LENGTH}; "
        f"H {r_xy:.4f} 0.0 {z_h:.4f}; "
        f"H {-r_xy/2:.4f} {r_xy * np.sqrt(3)/2:.4f} {z_h:.4f}; "
        f"H {-r_xy/2:.4f} {-r_xy * np.sqrt(3)/2:.4f} {z_h:.4f}"
    )

    # Dictionary to store results
    results = {}

    # --- 1. Classical Calculations (UHF and FCI) ---
    print("--- Running Classical Calculations ---")
    start_time = time.time()
    # Note: charge=0 and spin=0 for the closed-shell ground state of CH₄
    mol = gto.M(atom=atom_str, basis=BASIS_SET, charge=0, spin=0, verbose=0)
    
    # UHF Calculation
    mf_uhf = scf.UHF(mol).run()
    results['UHF'] = mf_uhf.e_tot
    print(f"UHF Energy: {results['UHF']:.8f} Hartree")
    
    # FCI Calculation (starting from RHF is standard for closed-shell systems)
    mf_rhf = scf.RHF(mol).run()
    results['FCI'] = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
    print(f"FCI (Exact) Energy: {results['FCI']:.8f} Hartree")
    print(f"Classical calculations finished in {time.time() - start_time:.2f}s\n")

    # --- 2. Quantum Calculations (VQE and AdaptVQE) ---
    
    # Set up the chemical problem for Qiskit, specifying charge=0 and spin=0
    driver = PySCFDriver(atom=atom_str, basis=BASIS_SET, charge=0, spin=0)
    problem = driver.run()
    mapper = JordanWignerMapper()

    # Define the UCCSD ansatz, which is used by both VQE methods
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
        ),
    )

    # Define the VQE instance that both algorithms will use
    vqe_instance = VQE(Estimator(), ansatz, SLSQP())
    vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

    # --- Standard VQE Calculation ---
    print("--- Running Standard VQE Calculation ---")
    start_time = time.time()
    solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
    result_vqe = solver_vqe.solve(problem)
    results['VQE'] = result_vqe.total_energies[0]
    print(f"Standard VQE Energy: {results['VQE']:.8f} Hartree")
    print(f"VQE finished in {time.time() - start_time:.2f}s\n")
    
    # --- AdaptVQE Calculation ---
    print("--- Running AdaptVQE Calculation ---")
    start_time = time.time()
    adapt_vqe = AdaptVQE(vqe_instance)
    # This temporary fix is needed for compatibility in some Qiskit versions
    adapt_vqe.supports_aux_operators = lambda: True
    
    solver_adapt = GroundStateEigensolver(mapper, adapt_vqe)
    result_adapt = solver_adapt.solve(problem)
    results['AdaptVQE'] = result_adapt.total_energies[0]
    print(f"AdaptVQE Energy: {results['AdaptVQE']:.8f} Hartree")
    print(f"AdaptVQE finished in {time.time() - start_time:.2f}s\n")

    # --- Final Summary ---
    print("\n" + "="*50)
    print("          Final Energy Summary")
    print("="*50)
    print(f"  UHF Energy:         {results['UHF']:.8f} Hartree")
    print(f"  Standard VQE Energy:  {results['VQE']:.8f} Hartree")
    print(f"  AdaptVQE Energy:      {results['AdaptVQE']:.8f} Hartree")
    print(f"  FCI (Exact) Energy:   {results['FCI']:.8f} Hartree")
    print("-" * 50)
    # Compare quantum results to the exact FCI energy
    print(f"  Error (VQE vs FCI):   {abs(results['VQE'] - results['FCI']):.8f} Hartree")
    print(f"  Error (AdaptVQE vs FCI): {abs(results['AdaptVQE'] - results['FCI']):.8f} Hartree")
    print("="*50)

if __name__ == "__main__":
    main()


CH₄ Ground State Calculation Comparison for Basis: sto-3g
Started at: 2025-09-09 13:58:18

--- Running Classical Calculations ---
UHF Energy: -39.72681100 Hartree
FCI (Exact) Energy: -39.80567659 Hartree
Classical calculations finished in 11.48s



/tmp/ipykernel_1012987/654117350.py:85: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_instance = VQE(Estimator(), ansatz, SLSQP())


--- Running Standard VQE Calculation ---


KeyboardInterrupt: 

In [2]:
import numpy as np
import time
from datetime import datetime
import warnings

# --- Suppress specific warnings for cleaner output ---
# The np.VisibleDeprecationWarning is removed as it's deprecated in newer NumPy versions
# Suppress potential future warnings from PySCF
warnings.filterwarnings("ignore", category=FutureWarning)


# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import (
    JordanWignerMapper,
    ParityMapper,
    BravyiKitaevMapper,
)
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA, SLSQP, SPSA
from qiskit.primitives import Estimator

# ==============================================================================
# ----- Configuration -----
# Here you can define all the components you want to test.
# ==============================================================================

BOND_LENGTH = 0.87  # Angstrom for H₃⁺ equilibrium distance

# --- 1. Choose your Basis Sets ---
# Note: More complex basis sets will significantly increase runtime.
BASIS_SETS = [
    "sto-3g",
    "3-21g",
    "6-31g",
    "6-31g*",
]

# --- 2. Choose your Fermion-to-Qubit Mappers ---
MAPPERS = {
    "Jordan-Wigner": JordanWignerMapper(),
    "Parity": ParityMapper(),
    "Bravyi-Kitaev": BravyiKitaevMapper(),
}

# --- 3. Choose your VQE Optimizers ---
OPTIMIZERS = {
    "COBYLA": COBYLA(maxiter=1000),
    "SLSQP": SLSQP(maxiter=1000),
    "SPSA": SPSA(maxiter=500),
}

# ==============================================================================
# ----- Main Calculation Function -----
# ==============================================================================

def main():
    """
    Calculates the ground state energy of H₃⁺ using a sweep of different
    basis sets, mappers, and optimizers, comparing quantum and classical methods.
    """
    print("=" * 80)
    print(" H₃⁺ Ground State Calculation Sweep")
    print(f" Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)

    # List to store all result dictionaries
    all_results = []
    
    # Define the molecule's geometry for H₃⁺ (equilateral triangle)
    # Coordinates are calculated to form a triangle with side BOND_LENGTH
    x_coord = BOND_LENGTH / 2.0
    y_coord = BOND_LENGTH * np.sqrt(3) / 2.0
    atom_str = f"H 0 0 0; H {BOND_LENGTH} 0 0; H {x_coord} {y_coord} 0"


    # --- Outer loop over Basis Sets ---
    for basis in BASIS_SETS:
        print(f"\n{'='*25} TESTING BASIS SET: {basis} {'='*25}\n")
        
        # --- 1. Classical Calculations (Done once per basis set) ---
        print(f"--- Running Classical Calculations for {basis} ---")
        classical_start_time = time.time()
        # Note: charge=1 and spin=0 for H₃⁺ (2 electrons)
        mol = gto.M(atom=atom_str, basis=basis, charge=1, spin=0, verbose=0)
        
        # UHF Calculation
        mf_uhf = scf.UHF(mol).run()
        uhf_energy = mf_uhf.e_tot
        
        # FCI (Exact) Calculation for reference
        mf_rhf = scf.RHF(mol).run()
        fci_energy = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
        
        print(f"UHF Energy: {uhf_energy:.8f} Hartree")
        print(f"FCI (Exact) Energy: {fci_energy:.8f} Hartree")
        print(f"Classical calculations finished in {time.time() - classical_start_time:.2f}s\n")

        # --- 2. Quantum Calculation Setup (Done once per basis set) ---
        # Note: charge=1 and spin=0 for H₃⁺
        driver = PySCFDriver(atom=atom_str, basis=basis, charge=1, spin=0)
        problem = driver.run()
        estimator = Estimator()

        # --- Inner loops for Mappers and Optimizers ---
        for mapper_name, mapper in MAPPERS.items():
            for optimizer_name, optimizer in OPTIMIZERS.items():
                
                config_id = (f"Basis: {basis}, Mapper: {mapper_name}, "
                             f"Optimizer: {optimizer_name}")
                print(f"--- Running VQE for: {config_id} ---")
                quantum_start_time = time.time()

                # The ansatz depends on the problem and mapper
                ansatz = UCCSD(
                    problem.num_spatial_orbitals,
                    problem.num_particles,
                    mapper,
                    initial_state=HartreeFock(
                        problem.num_spatial_orbitals,
                        problem.num_particles,
                        mapper,
                    ),
                )
                
                # Define the VQE instance for this run
                vqe_instance = VQE(estimator, ansatz, optimizer)
                vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

                # Standard VQE Calculation
                solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
                result_vqe = solver_vqe.solve(problem)
                vqe_total_energy = result_vqe.total_energies[0]
                
                print(f"Standard VQE Energy: {vqe_total_energy:.8f}")
                print(f"Quantum run finished in {time.time() - quantum_start_time:.2f}s\n")
                
                # Store results
                all_results.append({
                    "basis": basis,
                    "mapper": mapper_name,
                    "optimizer": optimizer_name,
                    "fci_energy": fci_energy,
                    "vqe_energy": vqe_total_energy,
                })

    # --- Final Summary Table ---
    print("\n" + "="*80)
    print(" " * 25 + "Final Comparison Summary")
    print("="*80)
    print(f"{'Basis Set':<12} | {'Mapper':<15} | {'Optimizer':<12} | {'VQE Energy':<18} | {'VQE Error':<15}")
    print("-" * 80)

    for res in all_results:
        fci_val = res['fci_energy']
        vqe = res['vqe_energy']
        vqe_error = abs(vqe - fci_val)
        
        print(f"{res['basis']:<12} | {res['mapper']:<15} | {res['optimizer']:<12} | "
              f"{vqe:<18.8f} | {vqe_error:<15.8f}")

    print("="*80)
    print(f"\nSweep completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


if __name__ == "__main__":
    main()



 H₃⁺ Ground State Calculation Sweep
 Started at: 2025-09-10 17:52:11

========================= TESTING BASIS SET: sto-3g =========================

--- Running Classical Calculations for sto-3g ---
UHF Energy: -1.23685875 Hartree
FCI (Exact) Energy: -1.26126403 Hartree
Classical calculations finished in 0.15s

--- Running VQE for: Basis: sto-3g, Mapper: Jordan-Wigner, Optimizer: COBYLA ---


/tmp/ipykernel_1588720/2421384838.py:110: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


Standard VQE Energy: -1.26126402
Quantum run finished in 11.48s

--- Running VQE for: Basis: sto-3g, Mapper: Jordan-Wigner, Optimizer: SLSQP ---
Standard VQE Energy: -1.26126402
Quantum run finished in 2.83s

--- Running VQE for: Basis: sto-3g, Mapper: Jordan-Wigner, Optimizer: SPSA ---
Standard VQE Energy: -1.26106892
Quantum run finished in 70.65s

--- Running VQE for: Basis: sto-3g, Mapper: Parity, Optimizer: COBYLA ---
Standard VQE Energy: -1.26126402
Quantum run finished in 9.73s

--- Running VQE for: Basis: sto-3g, Mapper: Parity, Optimizer: SLSQP ---
Standard VQE Energy: -1.26126402
Quantum run finished in 2.57s

--- Running VQE for: Basis: sto-3g, Mapper: Parity, Optimizer: SPSA ---
Standard VQE Energy: -1.24963494
Quantum run finished in 64.86s

--- Running VQE for: Basis: sto-3g, Mapper: Bravyi-Kitaev, Optimizer: COBYLA ---
Standard VQE Energy: -1.26126402
Quantum run finished in 11.08s

--- Running VQE for: Basis: sto-3g, Mapper: Bravyi-Kitaev, Optimizer: SLSQP ---
Standard 

/tmp/ipykernel_1588720/2421384838.py:110: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


--- Running VQE for: Basis: 3-21g, Mapper: Jordan-Wigner, Optimizer: COBYLA ---
Standard VQE Energy: -1.29972922
Quantum run finished in 419.66s

--- Running VQE for: Basis: 3-21g, Mapper: Jordan-Wigner, Optimizer: SLSQP ---
Standard VQE Energy: -1.29972912
Quantum run finished in 143.49s

--- Running VQE for: Basis: 3-21g, Mapper: Jordan-Wigner, Optimizer: SPSA ---
Standard VQE Energy: -0.63502407
Quantum run finished in 641.42s

--- Running VQE for: Basis: 3-21g, Mapper: Parity, Optimizer: COBYLA ---
Standard VQE Energy: -1.29972922
Quantum run finished in 528.74s

--- Running VQE for: Basis: 3-21g, Mapper: Parity, Optimizer: SLSQP ---
Standard VQE Energy: -1.29972912
Quantum run finished in 150.21s

--- Running VQE for: Basis: 3-21g, Mapper: Parity, Optimizer: SPSA ---
Standard VQE Energy: -0.62900163
Quantum run finished in 687.93s

--- Running VQE for: Basis: 3-21g, Mapper: Bravyi-Kitaev, Optimizer: COBYLA ---
Standard VQE Energy: -1.29972918
Quantum run finished in 450.47s

--- R

/tmp/ipykernel_1588720/2421384838.py:110: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


Standard VQE Energy: -1.30396616
Quantum run finished in 438.23s

--- Running VQE for: Basis: 6-31g, Mapper: Jordan-Wigner, Optimizer: SLSQP ---
Standard VQE Energy: -1.30396597
Quantum run finished in 168.37s

--- Running VQE for: Basis: 6-31g, Mapper: Jordan-Wigner, Optimizer: SPSA ---
Standard VQE Energy: -0.66567006
Quantum run finished in 644.17s

--- Running VQE for: Basis: 6-31g, Mapper: Parity, Optimizer: COBYLA ---
Standard VQE Energy: -1.30396616
Quantum run finished in 451.64s

--- Running VQE for: Basis: 6-31g, Mapper: Parity, Optimizer: SLSQP ---
Standard VQE Energy: -1.30396597
Quantum run finished in 174.09s

--- Running VQE for: Basis: 6-31g, Mapper: Parity, Optimizer: SPSA ---
Standard VQE Energy: 0.08635399
Quantum run finished in 692.77s

--- Running VQE for: Basis: 6-31g, Mapper: Bravyi-Kitaev, Optimizer: COBYLA ---
Standard VQE Energy: -1.30396617
Quantum run finished in 457.34s

--- Running VQE for: Basis: 6-31g, Mapper: Bravyi-Kitaev, Optimizer: SLSQP ---
Standar

/tmp/ipykernel_1588720/2421384838.py:110: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


Standard VQE Energy: -1.30396619
Quantum run finished in 452.27s

--- Running VQE for: Basis: 6-31g*, Mapper: Jordan-Wigner, Optimizer: SLSQP ---
Standard VQE Energy: -1.30396597
Quantum run finished in 169.63s

--- Running VQE for: Basis: 6-31g*, Mapper: Jordan-Wigner, Optimizer: SPSA ---
Standard VQE Energy: 0.09278828
Quantum run finished in 645.49s

--- Running VQE for: Basis: 6-31g*, Mapper: Parity, Optimizer: COBYLA ---
Standard VQE Energy: -1.30396615
Quantum run finished in 499.40s

--- Running VQE for: Basis: 6-31g*, Mapper: Parity, Optimizer: SLSQP ---
Standard VQE Energy: -1.30396597
Quantum run finished in 175.20s

--- Running VQE for: Basis: 6-31g*, Mapper: Parity, Optimizer: SPSA ---
Standard VQE Energy: -0.63090516
Quantum run finished in 689.96s

--- Running VQE for: Basis: 6-31g*, Mapper: Bravyi-Kitaev, Optimizer: COBYLA ---
Standard VQE Energy: -1.30396618
Quantum run finished in 576.25s

--- Running VQE for: Basis: 6-31g*, Mapper: Bravyi-Kitaev, Optimizer: SLSQP ---


In [3]:
import numpy as np
import time
from datetime import datetime
import warnings

# --- Suppress specific warnings for cleaner output ---
# The np.VisibleDeprecationWarning is removed as it's deprecated in newer NumPy versions
# Suppress potential future warnings from PySCF
warnings.filterwarnings("ignore", category=FutureWarning)


# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import (
    JordanWignerMapper,
    ParityMapper,
    BravyiKitaevMapper,
)
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator

# ==============================================================================
# ----- Configuration -----
# Here you can define all the components you want to test.
# ==============================================================================

BOND_LENGTH = 0.7414  # Angstrom for H₂ equilibrium distance

# --- 1. Choose your Basis Sets ---
# Note: More complex basis sets will significantly increase runtime.
BASIS_SETS = [
    "sto-3g",
    "3-21g",
    "6-31g",
    "6-31g*",
]

# --- 2. Choose your Fermion-to-Qubit Mappers ---
MAPPERS = {
    "Jordan-Wigner": JordanWignerMapper(),
    "Parity": ParityMapper(),
    "Bravyi-Kitaev": BravyiKitaevMapper(),
}

# --- 3. Choose your VQE Optimizers ---
OPTIMIZERS = {
    "L-BFGS-B": L_BFGS_B(maxiter=1000),
}

# ==============================================================================
# ----- Main Calculation Function -----
# ==============================================================================

def main():
    """
    Calculates the ground state energy of H₂ using a sweep of different
    basis sets and mappers with the L-BFGS-B optimizer.
    """
    print("=" * 80)
    print(" H₂ Ground State Calculation Sweep (Optimizer: L-BFGS-B)")
    print(f" Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)

    # List to store all result dictionaries
    all_results = []
    
    # Define the molecule's geometry for H₂
    atom_str = f"H 0 0 0; H 0 0 {BOND_LENGTH}"


    # --- Outer loop over Basis Sets ---
    for basis in BASIS_SETS:
        print(f"\n{'='*25} TESTING BASIS SET: {basis} {'='*25}\n")
        
        # --- 1. Classical Calculations (Done once per basis set) ---
        print(f"--- Running Classical Calculations for {basis} ---")
        classical_start_time = time.time()
        # Note: charge=0 and spin=0 for neutral H₂
        mol = gto.M(atom=atom_str, basis=basis, charge=0, spin=0, verbose=0)
        
        # UHF Calculation
        mf_uhf = scf.UHF(mol).run()
        uhf_energy = mf_uhf.e_tot
        
        # FCI (Exact) Calculation for reference
        mf_rhf = scf.RHF(mol).run()
        fci_energy = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
        
        print(f"UHF Energy: {uhf_energy:.8f} Hartree")
        print(f"FCI (Exact) Energy: {fci_energy:.8f} Hartree")
        print(f"Classical calculations finished in {time.time() - classical_start_time:.2f}s\n")

        # --- 2. Quantum Calculation Setup (Done once per basis set) ---
        # Note: charge=0 and spin=0 for neutral H₂
        driver = PySCFDriver(atom=atom_str, basis=basis, charge=0, spin=0)
        problem = driver.run()
        estimator = Estimator()

        # --- Inner loops for Mappers and Optimizers ---
        for mapper_name, mapper in MAPPERS.items():
            for optimizer_name, optimizer in OPTIMIZERS.items():
                
                config_id = (f"Basis: {basis}, Mapper: {mapper_name}, "
                             f"Optimizer: {optimizer_name}")
                print(f"--- Running VQE for: {config_id} ---")
                quantum_start_time = time.time()

                # The ansatz depends on the problem and mapper
                ansatz = UCCSD(
                    problem.num_spatial_orbitals,
                    problem.num_particles,
                    mapper,
                    initial_state=HartreeFock(
                        problem.num_spatial_orbitals,
                        problem.num_particles,
                        mapper,
                    ),
                )
                
                # Define the VQE instance for this run
                vqe_instance = VQE(estimator, ansatz, optimizer)
                vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

                # Standard VQE Calculation
                solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
                result_vqe = solver_vqe.solve(problem)
                vqe_total_energy = result_vqe.total_energies[0]
                
                print(f"Standard VQE Energy: {vqe_total_energy:.8f}")
                print(f"Quantum run finished in {time.time() - quantum_start_time:.2f}s\n")
                
                # Store results
                all_results.append({
                    "basis": basis,
                    "mapper": mapper_name,
                    "optimizer": optimizer_name,
                    "fci_energy": fci_energy,
                    "vqe_energy": vqe_total_energy,
                })

    # --- Final Summary Table ---
    print("\n" + "="*80)
    print(" " * 25 + "Final Comparison Summary")
    print("="*80)
    print(f"{'Basis Set':<12} | {'Mapper':<15} | {'Optimizer':<12} | {'VQE Energy':<18} | {'VQE Error':<15}")
    print("-" * 80)

    for res in all_results:
        fci_val = res['fci_energy']
        vqe = res['vqe_energy']
        vqe_error = abs(vqe - fci_val)
        
        print(f"{res['basis']:<12} | {res['mapper']:<15} | {res['optimizer']:<12} | "
              f"{vqe:<18.8f} | {vqe_error:<15.8f}")

    print("="*80)
    print(f"\nSweep completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


if __name__ == "__main__":
    main()



 H₂ Ground State Calculation Sweep (Optimizer: L-BFGS-B)
 Started at: 2025-09-10 21:18:11

========================= TESTING BASIS SET: sto-3g =========================

--- Running Classical Calculations for sto-3g ---
UHF Energy: -1.11668439 Hartree
FCI (Exact) Energy: -1.13727017 Hartree
Classical calculations finished in 0.17s



/tmp/ipykernel_1588720/2050528400.py:105: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


--- Running VQE for: Basis: sto-3g, Mapper: Jordan-Wigner, Optimizer: L-BFGS-B ---
Standard VQE Energy: -1.13727017
Quantum run finished in 0.83s

--- Running VQE for: Basis: sto-3g, Mapper: Parity, Optimizer: L-BFGS-B ---
Standard VQE Energy: -1.13727017
Quantum run finished in 0.75s

--- Running VQE for: Basis: sto-3g, Mapper: Bravyi-Kitaev, Optimizer: L-BFGS-B ---
Standard VQE Energy: -1.13727017
Quantum run finished in 0.74s


========================= TESTING BASIS SET: 3-21g =========================

--- Running Classical Calculations for 3-21g ---
UHF Energy: -1.12292833 Hartree
FCI (Exact) Energy: -1.14783032 Hartree
Classical calculations finished in 0.15s

--- Running VQE for: Basis: 3-21g, Mapper: Jordan-Wigner, Optimizer: L-BFGS-B ---
Standard VQE Energy: -1.14783032
Quantum run finished in 27.50s

--- Running VQE for: Basis: 3-21g, Mapper: Parity, Optimizer: L-BFGS-B ---
Standard VQE Energy: -1.14783032
Quantum run finished in 26.09s

--- Running VQE for: Basis: 3-21g, Ma

In [4]:
import numpy as np
import time
from datetime import datetime
import warnings

# --- Suppress specific warnings for cleaner output ---
# The np.VisibleDeprecationWarning is removed as it's deprecated in newer NumPy versions
# Suppress potential future warnings from PySCF
warnings.filterwarnings("ignore", category=FutureWarning)


# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import (
    JordanWignerMapper,
    ParityMapper,
    BravyiKitaevMapper,
)
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit.primitives import Estimator

# ==============================================================================
# ----- Configuration -----
# Here you can define all the components you want to test.
# ==============================================================================

BOND_LENGTH = 0.87  # Angstrom for H₃⁺ equilibrium distance

# --- 1. Choose your Basis Sets ---
# Note: More complex basis sets will significantly increase runtime.
BASIS_SETS = [
    "sto-3g",
    "3-21g",
    "6-31g",
    "6-31g*",
]

# --- 2. Choose your Fermion-to-Qubit Mappers ---
MAPPERS = {
    "Jordan-Wigner": JordanWignerMapper(),
    "Parity": ParityMapper(),
    "Bravyi-Kitaev": BravyiKitaevMapper(),
}

# --- 3. Choose your VQE Optimizers ---
OPTIMIZERS = {
    "L-BFGS-B": L_BFGS_B(maxiter=1000),
}

# ==============================================================================
# ----- Main Calculation Function -----
# ==============================================================================

def main():
    """
    Calculates the ground state energy of H₃⁺ using a sweep of different
    basis sets and mappers with the L-BFGS-B optimizer.
    """
    print("=" * 80)
    print(" H₃⁺ Ground State Calculation Sweep (Optimizer: L-BFGS-B)")
    print(f" Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)

    # List to store all result dictionaries
    all_results = []
    
    # Define the molecule's geometry for H₃⁺ (equilateral triangle)
    x_coord = BOND_LENGTH
    y_coord = BOND_LENGTH * np.sqrt(3) / 2
    atom_str = f"H 0 0 0; H {x_coord} 0 0; H {x_coord/2} {y_coord} 0"


    # --- Outer loop over Basis Sets ---
    for basis in BASIS_SETS:
        print(f"\n{'='*25} TESTING BASIS SET: {basis} {'='*25}\n")
        
        # --- 1. Classical Calculations (Done once per basis set) ---
        print(f"--- Running Classical Calculations for {basis} ---")
        classical_start_time = time.time()
        # Note: charge=1 and spin=0 for H₃⁺ cation
        mol = gto.M(atom=atom_str, basis=basis, charge=1, spin=0, verbose=0)
        
        # UHF Calculation
        mf_uhf = scf.UHF(mol).run()
        uhf_energy = mf_uhf.e_tot
        
        # FCI (Exact) Calculation for reference
        mf_rhf = scf.RHF(mol).run()
        fci_energy = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
        
        print(f"UHF Energy: {uhf_energy:.8f} Hartree")
        print(f"FCI (Exact) Energy: {fci_energy:.8f} Hartree")
        print(f"Classical calculations finished in {time.time() - classical_start_time:.2f}s\n")

        # --- 2. Quantum Calculation Setup (Done once per basis set) ---
        # Note: charge=1 and spin=0 for H₃⁺ cation
        driver = PySCFDriver(atom=atom_str, basis=basis, charge=1, spin=0)
        problem = driver.run()
        estimator = Estimator()

        # --- Inner loops for Mappers and Optimizers ---
        for mapper_name, mapper in MAPPERS.items():
            for optimizer_name, optimizer in OPTIMIZERS.items():
                
                config_id = (f"Basis: {basis}, Mapper: {mapper_name}, "
                             f"Optimizer: {optimizer_name}")
                print(f"--- Running VQE for: {config_id} ---")
                quantum_start_time = time.time()

                # The ansatz depends on the problem and mapper
                ansatz = UCCSD(
                    problem.num_spatial_orbitals,
                    problem.num_particles,
                    mapper,
                    initial_state=HartreeFock(
                        problem.num_spatial_orbitals,
                        problem.num_particles,
                        mapper,
                    ),
                )
                
                # Define the VQE instance for this run
                vqe_instance = VQE(estimator, ansatz, optimizer)
                vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

                # Standard VQE Calculation
                solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
                result_vqe = solver_vqe.solve(problem)
                vqe_total_energy = result_vqe.total_energies[0]
                
                print(f"Standard VQE Energy: {vqe_total_energy:.8f}")
                print(f"Quantum run finished in {time.time() - quantum_start_time:.2f}s\n")
                
                # Store results
                all_results.append({
                    "basis": basis,
                    "mapper": mapper_name,
                    "optimizer": optimizer_name,
                    "fci_energy": fci_energy,
                    "vqe_energy": vqe_total_energy,
                })

    # --- Final Summary Table ---
    print("\n" + "="*80)
    print(" " * 25 + "Final Comparison Summary")
    print("="*80)
    print(f"{'Basis Set':<12} | {'Mapper':<15} | {'Optimizer':<12} | {'VQE Energy':<18} | {'VQE Error':<15}")
    print("-" * 80)

    for res in all_results:
        fci_val = res['fci_energy']
        vqe = res['vqe_energy']
        vqe_error = abs(vqe - fci_val)
        
        print(f"{res['basis']:<12} | {res['mapper']:<15} | {res['optimizer']:<12} | "
              f"{vqe:<18.8f} | {vqe_error:<15.8f}")

    print("="*80)
    print(f"\nSweep completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


if __name__ == "__main__":
    main()



 H₃⁺ Ground State Calculation Sweep (Optimizer: L-BFGS-B)
 Started at: 2025-09-10 21:36:21

========================= TESTING BASIS SET: sto-3g =========================

--- Running Classical Calculations for sto-3g ---
UHF Energy: -1.23685875 Hartree
FCI (Exact) Energy: -1.26126403 Hartree
Classical calculations finished in 0.20s



/tmp/ipykernel_1588720/725262284.py:107: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


--- Running VQE for: Basis: sto-3g, Mapper: Jordan-Wigner, Optimizer: L-BFGS-B ---
Standard VQE Energy: -1.26126403
Quantum run finished in 4.93s

--- Running VQE for: Basis: sto-3g, Mapper: Parity, Optimizer: L-BFGS-B ---
Standard VQE Energy: -1.26126403
Quantum run finished in 4.29s

--- Running VQE for: Basis: sto-3g, Mapper: Bravyi-Kitaev, Optimizer: L-BFGS-B ---
Standard VQE Energy: -1.26126403
Quantum run finished in 4.76s


========================= TESTING BASIS SET: 3-21g =========================

--- Running Classical Calculations for 3-21g ---
UHF Energy: -1.26955353 Hartree
FCI (Exact) Energy: -1.29972927 Hartree
Classical calculations finished in 0.17s

--- Running VQE for: Basis: 3-21g, Mapper: Jordan-Wigner, Optimizer: L-BFGS-B ---
Standard VQE Energy: -1.29972927
Quantum run finished in 256.39s

--- Running VQE for: Basis: 3-21g, Mapper: Parity, Optimizer: L-BFGS-B ---
Standard VQE Energy: -1.29972927
Quantum run finished in 217.02s

--- Running VQE for: Basis: 3-21g, 

In [1]:
import numpy as np
import time
from datetime import datetime
import warnings

# --- Suppress specific warnings for cleaner output ---
# The np.VisibleDeprecationWarning is removed as it's deprecated in newer NumPy versions
# Suppress potential future warnings from PySCF
warnings.filterwarnings("ignore", category=FutureWarning)


# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import (
    JordanWignerMapper,
    ParityMapper,
    BravyiKitaevMapper,
)
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B, COBYLA, SLSQP, SPSA
from qiskit.primitives import Estimator

# ==============================================================================
# ----- Configuration -----
# Here you can define all the components you want to test.
# ==============================================================================

BOND_LENGTH = 1.595  # Angstrom for LiH equilibrium distance

# --- 1. Choose your Basis Sets ---
# Note: More complex basis sets will significantly increase runtime.
BASIS_SETS = [
    "sto-3g",
    "3-21g",
    "6-31g",
    "6-31g*",
]

# --- 2. Choose your Fermion-to-Qubit Mappers ---
MAPPERS = {
    "Jordan-Wigner": JordanWignerMapper(),
    "Parity": ParityMapper(),
    "Bravyi-Kitaev": BravyiKitaevMapper(),
}

# --- 3. Choose your VQE Optimizers ---
OPTIMIZERS = {
    "L-BFGS-B": L_BFGS_B(maxiter=1000),
    "COBYLA": COBYLA(maxiter=1000),
    "SLSQP": SLSQP(maxiter=1000),
    "SPSA": SPSA(maxiter=500),
}

# ==============================================================================
# ----- Main Calculation Function -----
# ==============================================================================

def main():
    """
    Calculates the ground state energy of LiH using a sweep of different
    basis sets, mappers, and optimizers.
    """
    print("=" * 80)
    print(" LiH Ground State Calculation Sweep")
    print(f" Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)

    # List to store all result dictionaries
    all_results = []
    
    # Define the molecule's geometry for LiH
    atom_str = f"Li 0 0 0; H 0 0 {BOND_LENGTH}"


    # --- Outer loop over Basis Sets ---
    for basis in BASIS_SETS:
        print(f"\n{'='*25} TESTING BASIS SET: {basis} {'='*25}\n")
        
        # --- 1. Classical Calculations (Done once per basis set) ---
        print(f"--- Running Classical Calculations for {basis} ---")
        classical_start_time = time.time()
        # Note: charge=0 and spin=0 for neutral LiH
        mol = gto.M(atom=atom_str, basis=basis, charge=0, spin=0, verbose=0)
        
        # UHF Calculation
        mf_uhf = scf.UHF(mol).run()
        uhf_energy = mf_uhf.e_tot
        
        # FCI (Exact) Calculation for reference
        mf_rhf = scf.RHF(mol).run()
        fci_energy = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
        
        print(f"UHF Energy: {uhf_energy:.8f} Hartree")
        print(f"FCI (Exact) Energy: {fci_energy:.8f} Hartree")
        print(f"Classical calculations finished in {time.time() - classical_start_time:.2f}s\n")

        # --- 2. Quantum Calculation Setup (Done once per basis set) ---
        # Note: charge=0 and spin=0 for neutral LiH
        driver = PySCFDriver(atom=atom_str, basis=basis, charge=0, spin=0)
        problem = driver.run()
        estimator = Estimator()

        # --- Inner loops for Mappers and Optimizers ---
        for mapper_name, mapper in MAPPERS.items():
            for optimizer_name, optimizer in OPTIMIZERS.items():
                
                config_id = (f"Basis: {basis}, Mapper: {mapper_name}, "
                             f"Optimizer: {optimizer_name}")
                print(f"--- Running VQE for: {config_id} ---")
                quantum_start_time = time.time()

                # The ansatz depends on the problem and mapper
                ansatz = UCCSD(
                    problem.num_spatial_orbitals,
                    problem.num_particles,
                    mapper,
                    initial_state=HartreeFock(
                        problem.num_spatial_orbitals,
                        problem.num_particles,
                        mapper,
                    ),
                )
                
                # Define the VQE instance for this run
                vqe_instance = VQE(estimator, ansatz, optimizer)
                vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

                # Standard VQE Calculation
                solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
                result_vqe = solver_vqe.solve(problem)
                vqe_total_energy = result_vqe.total_energies[0]
                
                print(f"Standard VQE Energy: {vqe_total_energy:.8f}")
                print(f"Quantum run finished in {time.time() - quantum_start_time:.2f}s\n")
                
                # Store results
                all_results.append({
                    "basis": basis,
                    "mapper": mapper_name,
                    "optimizer": optimizer_name,
                    "fci_energy": fci_energy,
                    "vqe_energy": vqe_total_energy,
                })

    # --- Final Summary Table ---
    print("\n" + "="*80)
    print(" " * 25 + "Final Comparison Summary")
    print("="*80)
    print(f"{'Basis Set':<12} | {'Mapper':<15} | {'Optimizer':<12} | {'VQE Energy':<18} | {'VQE Error':<15}")
    print("-" * 80)

    for res in all_results:
        fci_val = res['fci_energy']
        vqe = res['vqe_energy']
        vqe_error = abs(vqe - fci_val)
        
        print(f"{res['basis']:<12} | {res['mapper']:<15} | {res['optimizer']:<12} | "
              f"{vqe:<18.8f} | {vqe_error:<15.8f}")

    print("="*80)
    print(f"\nSweep completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


if __name__ == "__main__":
    main()



 LiH Ground State Calculation Sweep
 Started at: 2025-09-10 23:04:08

========================= TESTING BASIS SET: sto-3g =========================

--- Running Classical Calculations for sto-3g ---
UHF Energy: -7.86202386 Hartree
FCI (Exact) Energy: -7.88240193 Hartree
Classical calculations finished in 0.59s



/tmp/ipykernel_1646462/1893684782.py:108: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


--- Running VQE for: Basis: sto-3g, Mapper: Jordan-Wigner, Optimizer: L-BFGS-B ---
Standard VQE Energy: -7.88239129
Quantum run finished in 5579.83s

--- Running VQE for: Basis: sto-3g, Mapper: Jordan-Wigner, Optimizer: COBYLA ---
Standard VQE Energy: -7.88079950
Quantum run finished in 1676.57s

--- Running VQE for: Basis: sto-3g, Mapper: Jordan-Wigner, Optimizer: SLSQP ---
Standard VQE Energy: -7.88239110
Quantum run finished in 1670.92s

--- Running VQE for: Basis: sto-3g, Mapper: Jordan-Wigner, Optimizer: SPSA ---
Standard VQE Energy: -7.10409985
Quantum run finished in 1715.54s

--- Running VQE for: Basis: sto-3g, Mapper: Parity, Optimizer: L-BFGS-B ---
Standard VQE Energy: -7.88239129
Quantum run finished in 6086.98s

--- Running VQE for: Basis: sto-3g, Mapper: Parity, Optimizer: COBYLA ---
Standard VQE Energy: -7.88135145
Quantum run finished in 1776.13s

--- Running VQE for: Basis: sto-3g, Mapper: Parity, Optimizer: SLSQP ---
Standard VQE Energy: -7.88239110
Quantum run finishe

/tmp/ipykernel_1646462/1893684782.py:108: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


KeyboardInterrupt: 

In [1]:
import numpy as np
import time
from datetime import datetime
import warnings

# --- Suppress specific warnings for cleaner output ---
# The np.VisibleDeprecationWarning is removed as it's deprecated in newer NumPy versions
# Suppress potential future warnings from PySCF
warnings.filterwarnings("ignore", category=FutureWarning)


# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import (
    JordanWignerMapper,
    ParityMapper,
    BravyiKitaevMapper,
)
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B, COBYLA, SLSQP, SPSA
from qiskit.primitives import Estimator

# ==============================================================================
# ----- Configuration -----
# Here you can define all the components you want to test.
# ==============================================================================

BOND_LENGTH = 1.595  # Angstrom for LiH equilibrium distance

# --- 1. Choose your Basis Sets ---
# Restoring the full list for a comprehensive sweep.
BASIS_SETS = [
    "3-21g",
    "6-31g",
    "6-31g*",
]

# --- 2. Choose your Fermion-to-Qubit Mappers ---
# Restoring the full list of mappers.
MAPPERS = {
    "Jordan-Wigner": JordanWignerMapper(),
    "Parity": ParityMapper(),
    "Bravyi-Kitaev": BravyiKitaevMapper(),
}

# --- 3. Choose your VQE Optimizers ---
# Restoring the full list of optimizers with standard iterations.
OPTIMIZERS = {
    "L-BFGS-B": L_BFGS_B(maxiter=1000),
    "COBYLA": COBYLA(maxiter=1000),
    "SLSQP": SLSQP(maxiter=1000),
    "SPSA": SPSA(maxiter=500),
}

# ==============================================================================
# ----- Main Calculation Function -----
# ==============================================================================

def main():
    """
    Calculates the ground state energy of LiH using a sweep of different
    basis sets, mappers, and optimizers, with a specific FreezeCoreTransformer.
    """
    print("=" * 80)
    print(" LiH Ground State Calculation Sweep (with Advanced FreezeCore)")
    print(f" Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)

    # List to store all result dictionaries
    all_results = []
    
    # Define the molecule's geometry for LiH
    atom_str = f"Li 0 0 0; H 0 0 {BOND_LENGTH}"


    # --- Outer loop over Basis Sets ---
    for basis in BASIS_SETS:
        print(f"\n{'='*25} TESTING BASIS SET: {basis} {'='*25}\n")
        
        # --- 1. Classical Calculations (Done once per basis set) ---
        print(f"--- Running Classical Calculations for {basis} ---")
        classical_start_time = time.time()
        # Note: charge=0 and spin=0 for neutral LiH
        mol = gto.M(atom=atom_str, basis=basis, charge=0, spin=0, verbose=0)
        
        # UHF Calculation
        mf_uhf = scf.UHF(mol).run()
        uhf_energy = mf_uhf.e_tot
        
        # FCI (Exact) Calculation for reference
        mf_rhf = scf.RHF(mol).run()
        fci_energy = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
        
        print(f"UHF Energy: {uhf_energy:.8f} Hartree")
        print(f"FCI (Exact) Energy: {fci_energy:.8f} Hartree")
        print(f"Classical calculations finished in {time.time() - classical_start_time:.2f}s\n")

        # --- 2. Quantum Calculation Setup (Done once per basis set) ---
        # Note: charge=0 and spin=0 for neutral LiH
        driver = PySCFDriver(atom=atom_str, basis=basis, charge=0, spin=0)
        problem = driver.run()

        # --- 3. Apply the FreezeCoreTransformer with specific arguments ---
        # This freezes the core (Lithium 1s) and manually removes two
        # additional orbitals to create a smaller, custom active space.
        print("--- Applying FreezeCoreTransformer with specific arguments ---")
        transformer = FreezeCoreTransformer(freeze_core=True, remove_orbitals=[-3, -2])
        problem = transformer.transform(problem)
        print(f"Problem transformed. Active electrons: {problem.num_particles}")
        print(f"Active spatial orbitals: {problem.num_spatial_orbitals}\n")

        estimator = Estimator()

        # --- Inner loops for Mappers and Optimizers ---
        for mapper_name, mapper in MAPPERS.items():
            for optimizer_name, optimizer in OPTIMIZERS.items():
                
                config_id = (f"Basis: {basis}, Mapper: {mapper_name}, "
                             f"Optimizer: {optimizer_name}")
                print(f"--- Running VQE for: {config_id} ---")
                quantum_start_time = time.time()

                # The ansatz depends on the transformed problem and mapper
                ansatz = UCCSD(
                    problem.num_spatial_orbitals,
                    problem.num_particles,
                    mapper,
                    initial_state=HartreeFock(
                        problem.num_spatial_orbitals,
                        problem.num_particles,
                        mapper,
                    ),
                )
                
                # Define the VQE instance for this run
                vqe_instance = VQE(estimator, ansatz, optimizer)
                vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

                # Standard VQE Calculation
                solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
                result_vqe = solver_vqe.solve(problem)
                vqe_total_energy = result_vqe.total_energies[0]
                
                print(f"Standard VQE Energy: {vqe_total_energy:.8f}")
                print(f"Quantum run finished in {time.time() - quantum_start_time:.2f}s\n")
                
                # Store results
                all_results.append({
                    "basis": basis,
                    "mapper": mapper_name,
                    "optimizer": optimizer_name,
                    "fci_energy": fci_energy,
                    "vqe_energy": vqe_total_energy,
                })

    # --- Final Summary Table ---
    print("\n" + "="*80)
    print(" " * 25 + "Final Comparison Summary")
    print("="*80)
    print(f"{'Basis Set':<12} | {'Mapper':<15} | {'Optimizer':<12} | {'VQE Energy':<18} | {'VQE Error':<15}")
    print("-" * 80)

    for res in all_results:
        fci_val = res['fci_energy']
        vqe = res['vqe_energy']
        vqe_error = abs(vqe - fci_val)
        
        print(f"{res['basis']:<12} | {res['mapper']:<15} | {res['optimizer']:<12} | "
              f"{vqe:<18.8f} | {vqe_error:<15.8f}")

    print("="*80)
    print(f"\nSweep completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


if __name__ == "__main__":
    main()



 LiH Ground State Calculation Sweep (with Advanced FreezeCore)
 Started at: 2025-09-11 11:49:18

========================= TESTING BASIS SET: 3-21g =========================

--- Running Classical Calculations for 3-21g ---
UHF Energy: -7.92958651 Hartree
FCI (Exact) Energy: -7.94868759 Hartree
Classical calculations finished in 0.66s

--- Applying FreezeCoreTransformer with specific arguments ---
Problem transformed. Active electrons: (1, 1)
Active spatial orbitals: 10

--- Running VQE for: Basis: 3-21g, Mapper: Jordan-Wigner, Optimizer: L-BFGS-B ---


/tmp/ipykernel_1901720/265919806.py:120: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


KeyboardInterrupt: 

In [1]:
import numpy as np
import time
from datetime import datetime
import warnings

# --- Suppress specific warnings for cleaner output ---
# The np.VisibleDeprecationWarning is removed as it's deprecated in newer NumPy versions
# Suppress potential future warnings from PySCF
warnings.filterwarnings("ignore", category=FutureWarning)


# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import (
    JordanWignerMapper,
    ParityMapper,
    BravyiKitaevMapper,
)
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer # <-- IMPORT ADDED

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B, COBYLA, SLSQP, SPSA
from qiskit.primitives import Estimator

# ==============================================================================
# ----- Configuration -----
# Here you can define all the components you want to test.
# ==============================================================================

BOND_LENGTH = 0.917  # Angstrom for HF equilibrium distance

# --- 1. Choose your Basis Sets ---
# Note: More complex basis sets will significantly increase runtime.
BASIS_SETS = [
    "3-21g",
    "6-31g",
    "6-31g*",
]

# --- 2. Choose your Fermion-to-Qubit Mappers ---
MAPPERS = {
    "Jordan-Wigner": JordanWignerMapper(),
    "Parity": ParityMapper(),
    "Bravyi-Kitaev": BravyiKitaevMapper(),
}

# --- 3. Choose your VQE Optimizers ---
OPTIMIZERS = {
    "L-BFGS-B": L_BFGS_B(maxiter=1000),
    "COBYLA": COBYLA(maxiter=1000),
    "SLSQP": SLSQP(maxiter=1000),
    "SPSA": SPSA(maxiter=500),
}

# ==============================================================================
# ----- Main Calculation Function -----
# ==============================================================================

def main():
    """
    Calculates the ground state energy of HF using a sweep of different
    basis sets, mappers, and optimizers, with a FreezeCoreTransformer.
    """
    print("=" * 80)
    print(" HF Ground State Calculation Sweep (with FreezeCoreTransformer)")
    print(f" Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)

    # List to store all result dictionaries
    all_results = []
    
    # Define the molecule's geometry for HF
    atom_str = f"H 0 0 0; F 0 0 {BOND_LENGTH}"


    # --- Outer loop over Basis Sets ---
    for basis in BASIS_SETS:
        print(f"\n{'='*25} TESTING BASIS SET: {basis} {'='*25}\n")
        
        # --- 1. Classical Calculations (Done once per basis set) ---
        print(f"--- Running Classical Calculations for {basis} ---")
        classical_start_time = time.time()
        # Note: charge=0 and spin=0 for neutral HF
        mol = gto.M(atom=atom_str, basis=basis, charge=0, spin=0, verbose=0)
        
        # UHF Calculation
        mf_uhf = scf.UHF(mol).run()
        uhf_energy = mf_uhf.e_tot
        
        # FCI (Exact) Calculation for reference
        mf_rhf = scf.RHF(mol).run()
        fci_energy = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
        
        print(f"UHF Energy: {uhf_energy:.8f} Hartree")
        print(f"FCI (Exact) Energy: {fci_energy:.8f} Hartree")
        print(f"Classical calculations finished in {time.time() - classical_start_time:.2f}s\n")

        # --- 2. Quantum Calculation Setup (Done once per basis set) ---
        # Note: charge=0 and spin=0 for neutral HF
        driver = PySCFDriver(atom=atom_str, basis=basis, charge=0, spin=0)
        problem = driver.run()

        # --- 3. Apply the FreezeCoreTransformer ---
        # This reduces the problem size by removing core electrons/orbitals.
        print("--- Applying FreezeCoreTransformer ---")
        transformer = FreezeCoreTransformer()
        problem = transformer.transform(problem)
        print(f"Problem transformed. Active electrons: {problem.num_particles}")
        print(f"Active spatial orbitals: {problem.num_spatial_orbitals}\n")

        estimator = Estimator()

        # --- Inner loops for Mappers and Optimizers ---
        for mapper_name, mapper in MAPPERS.items():
            for optimizer_name, optimizer in OPTIMIZERS.items():
                
                config_id = (f"Basis: {basis}, Mapper: {mapper_name}, "
                             f"Optimizer: {optimizer_name}")
                print(f"--- Running VQE for: {config_id} ---")
                quantum_start_time = time.time()

                # The ansatz depends on the problem and mapper
                ansatz = UCCSD(
                    problem.num_spatial_orbitals,
                    problem.num_particles,
                    mapper,
                    initial_state=HartreeFock(
                        problem.num_spatial_orbitals,
                        problem.num_particles,
                        mapper,
                    ),
                )
                
                # Define the VQE instance for this run
                vqe_instance = VQE(estimator, ansatz, optimizer)
                vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

                # Standard VQE Calculation
                solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
                result_vqe = solver_vqe.solve(problem)
                vqe_total_energy = result_vqe.total_energies[0]
                
                print(f"Standard VQE Energy: {vqe_total_energy:.8f}")
                print(f"Quantum run finished in {time.time() - quantum_start_time:.2f}s\n")
                
                # Store results
                all_results.append({
                    "basis": basis,
                    "mapper": mapper_name,
                    "optimizer": optimizer_name,
                    "fci_energy": fci_energy,
                    "vqe_energy": vqe_total_energy,
                })

    # --- Final Summary Table ---
    print("\n" + "="*80)
    print(" " * 25 + "Final Comparison Summary")
    print("="*80)
    print(f"{'Basis Set':<12} | {'Mapper':<15} | {'Optimizer':<12} | {'VQE Energy':<18} | {'VQE Error':<15}")
    print("-" * 80)

    for res in all_results:
        fci_val = res['fci_energy']
        vqe = res['vqe_energy']
        vqe_error = abs(vqe - fci_val)
        
        print(f"{res['basis']:<12} | {res['mapper']:<15} | {res['optimizer']:<12} | "
              f"{vqe:<18.8f} | {vqe_error:<15.8f}")

    print("="*80)
    print(f"\nSweep completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


if __name__ == "__main__":
    main()



 HF Ground State Calculation Sweep (with FreezeCoreTransformer)
 Started at: 2025-09-11 11:37:57

========================= TESTING BASIS SET: 3-21g =========================

--- Running Classical Calculations for 3-21g ---
UHF Energy: -99.45975177 Hartree
FCI (Exact) Energy: -99.58644769 Hartree
Classical calculations finished in 1.91s

--- Applying FreezeCoreTransformer ---
Problem transformed. Active electrons: (4, 4)
Active spatial orbitals: 10

--- Running VQE for: Basis: 3-21g, Mapper: Jordan-Wigner, Optimizer: L-BFGS-B ---


/tmp/ipykernel_1881219/2270134895.py:117: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


KeyboardInterrupt: 

In [1]:
import numpy as np
import time
from datetime import datetime
import warnings

# --- Suppress specific warnings for cleaner output ---
# The np.VisibleDeprecationWarning is removed as it's deprecated in newer NumPy versions
# Suppress potential future warnings from PySCF
warnings.filterwarnings("ignore", category=FutureWarning)


# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import (
    JordanWignerMapper,
    ParityMapper,
    BravyiKitaevMapper,
)
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B, COBYLA, SLSQP, SPSA
from qiskit.primitives import Estimator

# ==============================================================================
# ----- Configuration -----
# Here you can define all the components you want to test.
# ==============================================================================

BOND_LENGTH = 0.917  # Angstrom for HF equilibrium distance

# --- 1. Choose your Basis Sets ---
# Note: More complex basis sets will significantly increase runtime.
BASIS_SETS = [
    "3-21g",
    "6-31g",
    "6-31g*",
]

# --- 2. Choose your Fermion-to-Qubit Mappers ---
MAPPERS = {
    "Jordan-Wigner": JordanWignerMapper(),
    "Parity": ParityMapper(),
    "Bravyi-Kitaev": BravyiKitaevMapper(),
}

# --- 3. Choose your VQE Optimizers ---
OPTIMIZERS = {
    "L-BFGS-B": L_BFGS_B(maxiter=1000),
    "COBYLA": COBYLA(maxiter=1000),
    "SLSQP": SLSQP(maxiter=1000),
    "SPSA": SPSA(maxiter=500),
}

# ==============================================================================
# ----- Main Calculation Function -----
# ==============================================================================

def main():
    """
    Calculates the ground state energy of HF using a sweep of different
    basis sets, mappers, and optimizers, with an aggressive FreezeCoreTransformer.
    """
    print("=" * 80)
    print(" HF Ground State Calculation Sweep (with Smaller Active Space)")
    print(f" Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)

    # List to store all result dictionaries
    all_results = []
    
    # Define the molecule's geometry for HF
    atom_str = f"H 0 0 0; F 0 0 {BOND_LENGTH}"


    # --- Outer loop over Basis Sets ---
    for basis in BASIS_SETS:
        print(f"\n{'='*25} TESTING BASIS SET: {basis} {'='*25}\n")
        
        # --- 1. Classical Calculations (Done once per basis set) ---
        print(f"--- Running Classical Calculations for {basis} ---")
        classical_start_time = time.time()
        # Note: charge=0 and spin=0 for neutral HF
        mol = gto.M(atom=atom_str, basis=basis, charge=0, spin=0, verbose=0)
        
        # UHF Calculation
        mf_uhf = scf.UHF(mol).run()
        uhf_energy = mf_uhf.e_tot
        
        # FCI (Exact) Calculation for reference
        mf_rhf = scf.RHF(mol).run()
        fci_energy = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]
        
        print(f"UHF Energy: {uhf_energy:.8f} Hartree")
        print(f"FCI (Exact) Energy: {fci_energy:.8f} Hartree")
        print(f"Classical calculations finished in {time.time() - classical_start_time:.2f}s\n")

        # --- 2. Quantum Calculation Setup (Done once per basis set) ---
        # Note: charge=0 and spin=0 for neutral HF
        driver = PySCFDriver(atom=atom_str, basis=basis, charge=0, spin=0)
        problem = driver.run()

        # --- 3. Apply the FreezeCoreTransformer with an additional argument ---
        # This freezes the core (Fluorine 1s) AND removes the highest-energy
        # virtual orbital to speed up the calculation.
        print("--- Applying FreezeCoreTransformer to reduce active space ---")
        transformer = FreezeCoreTransformer(remove_orbitals=[-1])
        problem = transformer.transform(problem)
        print(f"Problem transformed. Active electrons: {problem.num_particles}")
        print(f"Active spatial orbitals: {problem.num_spatial_orbitals}\n")

        estimator = Estimator()

        # --- Inner loops for Mappers and Optimizers ---
        for mapper_name, mapper in MAPPERS.items():
            for optimizer_name, optimizer in OPTIMIZERS.items():
                
                config_id = (f"Basis: {basis}, Mapper: {mapper_name}, "
                             f"Optimizer: {optimizer_name}")
                print(f"--- Running VQE for: {config_id} ---")
                quantum_start_time = time.time()

                # The ansatz depends on the transformed problem and mapper
                ansatz = UCCSD(
                    problem.num_spatial_orbitals,
                    problem.num_particles,
                    mapper,
                    initial_state=HartreeFock(
                        problem.num_spatial_orbitals,
                        problem.num_particles,
                        mapper,
                    ),
                )
                
                # Define the VQE instance for this run
                vqe_instance = VQE(estimator, ansatz, optimizer)
                vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

                # Standard VQE Calculation
                solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
                result_vqe = solver_vqe.solve(problem)
                vqe_total_energy = result_vqe.total_energies[0]
                
                print(f"Standard VQE Energy: {vqe_total_energy:.8f}")
                print(f"Quantum run finished in {time.time() - quantum_start_time:.2f}s\n")
                
                # Store results
                all_results.append({
                    "basis": basis,
                    "mapper": mapper_name,
                    "optimizer": optimizer_name,
                    "fci_energy": fci_energy,
                    "vqe_energy": vqe_total_energy,
                })

    # --- Final Summary Table ---
    print("\n" + "="*80)
    print(" " * 25 + "Final Comparison Summary")
    print("="*80)
    print(f"{'Basis Set':<12} | {'Mapper':<15} | {'Optimizer':<12} | {'VQE Energy':<18} | {'VQE Error':<15}")
    print("-" * 80)

    for res in all_results:
        fci_val = res['fci_energy']
        vqe = res['vqe_energy']
        vqe_error = abs(vqe - fci_val)
        
        print(f"{res['basis']:<12} | {res['mapper']:<15} | {res['optimizer']:<12} | "
              f"{vqe:<18.8f} | {vqe_error:<15.8f}")

    print("="*80)
    print(f"\nSweep completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


if __name__ == "__main__":
    main()



 HF Ground State Calculation Sweep (with Smaller Active Space)
 Started at: 2025-09-11 11:43:18

========================= TESTING BASIS SET: 3-21g =========================

--- Running Classical Calculations for 3-21g ---
UHF Energy: -99.45975177 Hartree
FCI (Exact) Energy: -99.58644769 Hartree
Classical calculations finished in 1.46s

--- Applying FreezeCoreTransformer to reduce active space ---
Problem transformed. Active electrons: (4, 4)
Active spatial orbitals: 10

--- Running VQE for: Basis: 3-21g, Mapper: Jordan-Wigner, Optimizer: L-BFGS-B ---


/tmp/ipykernel_1899500/1817460426.py:118: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


KeyboardInterrupt: 

In [1]:
import numpy as np
import time
from datetime import datetime
import warnings

# --- Suppress specific warnings for cleaner output ---
# The np.VisibleDeprecationWarning is removed as it's deprecated in newer NumPy versions
# Suppress potential future warnings from PySCF
warnings.filterwarnings("ignore", category=FutureWarning)


# --- Classical Chemistry Imports ---
from pyscf import gto, scf, fci

# --- Qiskit Nature Imports ---
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import (
    JordanWignerMapper,
    ParityMapper,
    BravyiKitaevMapper,
)
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

# --- Qiskit Algorithm Imports ---
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B, COBYLA, SLSQP, SPSA
from qiskit.primitives import Estimator

# ==============================================================================
# ----- Configuration -----
# Here you can define all the components you want to test.
# ==============================================================================

BOND_LENGTH = 0.97  # Angstrom for OH- equilibrium distance

# --- 1. Choose your Basis Sets ---
# Note: More complex basis sets will significantly increase runtime.
BASIS_SETS = [
    "sto-3g",
    "3-21g",
    "6-31g",
    "6-31g*",
]

# --- 2. Choose your Fermion-to-Qubit Mappers ---
MAPPERS = {
    "Jordan-Wigner": JordanWignerMapper(),
    "Parity": ParityMapper(),
    "Bravyi-Kitaev": BravyiKitaevMapper(),
}

# --- 3. Choose your VQE Optimizers ---
OPTIMIZERS = {
    "L-BFGS-B": L_BFGS_B(maxiter=1000),
    "COBYLA": COBYLA(maxiter=1000),
    "SLSQP": SLSQP(maxiter=1000),
    "SPSA": SPSA(maxiter=500),
}

# ==============================================================================
# ----- Main Calculation Function -----
# ==============================================================================

def main():
    """
    Calculates the ground state energy of OH- using a sweep of different
    basis sets, mappers, and optimizers, with a FreezeCoreTransformer.
    """
    print("=" * 80)
    print(" OH- Ground State Calculation Sweep (with FreezeCoreTransformer)")
    print(f" Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)

    # List to store all result dictionaries
    all_results = []

    # Define the molecule's geometry for OH-
    atom_str = f"O 0 0 0; H 0 0 {BOND_LENGTH}"


    # --- Outer loop over Basis Sets ---
    for basis in BASIS_SETS:
        print(f"\n{'='*25} TESTING BASIS SET: {basis} {'='*25}\n")

        # --- 1. Classical Calculations (Done once per basis set) ---
        print(f"--- Running Classical Calculations for {basis} ---")
        classical_start_time = time.time()
        # Note: charge=-1 and spin=0 for the hydroxide anion (OH-)
        mol = gto.M(atom=atom_str, basis=basis, charge=-1, spin=0, verbose=0)

        # UHF Calculation
        mf_uhf = scf.UHF(mol).run()
        uhf_energy = mf_uhf.e_tot

        # FCI (Exact) Calculation for reference
        mf_rhf = scf.RHF(mol).run()
        fci_energy = fci.FCI(mol, mf_rhf.mo_coeff).kernel()[0]

        print(f"UHF Energy: {uhf_energy:.8f} Hartree")
        print(f"FCI (Exact) Energy: {fci_energy:.8f} Hartree")
        print(f"Classical calculations finished in {time.time() - classical_start_time:.2f}s\n")

        # --- 2. Quantum Calculation Setup (Done once per basis set) ---
        # Note: charge=-1 and spin=0 for the hydroxide anion (OH-)
        driver = PySCFDriver(atom=atom_str, basis=basis, charge=-1, spin=0)
        problem = driver.run()

        # --- 3. Apply the FreezeCoreTransformer ---
        # We call it without arguments to let it auto-detect the core orbitals.
        # For OH-, it will correctly identify and freeze the 1s orbital of Oxygen.
        print("--- Applying FreezeCoreTransformer ---")
        transformer = FreezeCoreTransformer()
        problem = transformer.transform(problem)
        print(f"Problem transformed. Active electrons: {problem.num_particles}")
        print(f"Active spatial orbitals: {problem.num_spatial_orbitals}\n")

        estimator = Estimator()

        # --- Inner loops for Mappers and Optimizers ---
        for mapper_name, mapper in MAPPERS.items():
            for optimizer_name, optimizer in OPTIMIZERS.items():

                config_id = (f"Basis: {basis}, Mapper: {mapper_name}, "
                             f"Optimizer: {optimizer_name}")
                print(f"--- Running VQE for: {config_id} ---")
                quantum_start_time = time.time()

                # The ansatz depends on the transformed problem and mapper
                ansatz = UCCSD(
                    problem.num_spatial_orbitals,
                    problem.num_particles,
                    mapper,
                    initial_state=HartreeFock(
                        problem.num_spatial_orbitals,
                        problem.num_particles,
                        mapper,
                    ),
                )

                # Define the VQE instance for this run
                vqe_instance = VQE(estimator, ansatz, optimizer)
                vqe_instance.initial_point = np.zeros(ansatz.num_parameters)

                # Standard VQE Calculation
                solver_vqe = GroundStateEigensolver(mapper, vqe_instance)
                result_vqe = solver_vqe.solve(problem)
                vqe_total_energy = result_vqe.total_energies[0]

                print(f"Standard VQE Energy: {vqe_total_energy:.8f}")
                print(f"Quantum run finished in {time.time() - quantum_start_time:.2f}s\n")

                # Store results
                all_results.append({
                    "basis": basis,
                    "mapper": mapper_name,
                    "optimizer": optimizer_name,
                    "fci_energy": fci_energy,
                    "vqe_energy": vqe_total_energy,
                })

    # --- Final Summary Table ---
    print("\n" + "="*80)
    print(" " * 25 + "Final Comparison Summary")
    print("="*80)
    print(f"{'Basis Set':<12} | {'Mapper':<15} | {'Optimizer':<12} | {'VQE Energy':<18} | {'VQE Error':<15}")
    print("-" * 80)

    for res in all_results:
        fci_val = res['fci_energy']
        vqe = res['vqe_energy']
        vqe_error = abs(vqe - fci_val)

        print(f"{res['basis']:<12} | {res['mapper']:<15} | {res['optimizer']:<12} | "
              f"{vqe:<18.8f} | {vqe_error:<15.8f}")

    print("="*80)
    print(f"\nSweep completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


if __name__ == "__main__":
    main()



 OH- Ground State Calculation Sweep (with FreezeCoreTransformer)
 Started at: 2025-09-11 11:53:57

========================= TESTING BASIS SET: sto-3g =========================

--- Running Classical Calculations for sto-3g ---
UHF Energy: -74.05739919 Hartree
FCI (Exact) Energy: -74.08172336 Hartree
Classical calculations finished in 0.86s

--- Applying FreezeCoreTransformer ---
Problem transformed. Active electrons: (4, 4)
Active spatial orbitals: 5

--- Running VQE for: Basis: sto-3g, Mapper: Jordan-Wigner, Optimizer: L-BFGS-B ---


/tmp/ipykernel_1902343/1754282270.py:119: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


Standard VQE Energy: -74.08168288
Quantum run finished in 92.74s

--- Running VQE for: Basis: sto-3g, Mapper: Jordan-Wigner, Optimizer: COBYLA ---
Standard VQE Energy: -74.08168285
Quantum run finished in 163.86s

--- Running VQE for: Basis: sto-3g, Mapper: Jordan-Wigner, Optimizer: SLSQP ---
Standard VQE Energy: -74.08168244
Quantum run finished in 41.45s

--- Running VQE for: Basis: sto-3g, Mapper: Jordan-Wigner, Optimizer: SPSA ---
Standard VQE Energy: -74.06422434
Quantum run finished in 313.77s

--- Running VQE for: Basis: sto-3g, Mapper: Parity, Optimizer: L-BFGS-B ---
Standard VQE Energy: -74.08168288
Quantum run finished in 72.38s

--- Running VQE for: Basis: sto-3g, Mapper: Parity, Optimizer: COBYLA ---
Standard VQE Energy: -74.08168284
Quantum run finished in 149.61s

--- Running VQE for: Basis: sto-3g, Mapper: Parity, Optimizer: SLSQP ---
Standard VQE Energy: -74.08168245
Quantum run finished in 42.94s

--- Running VQE for: Basis: sto-3g, Mapper: Parity, Optimizer: SPSA ---


/tmp/ipykernel_1902343/1754282270.py:119: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


KeyboardInterrupt: 